In [1]:
import psycopg2
import pandas as pd
import numpy as np
from jinja2 import Environment, FileSystemLoader
from config import config

params = config()
conn = psycopg2.connect(**params)

zona = 'tornavacas'
comarca = 'jerte'

capitulos = [
    ('Objeto y ámbito legal', 'objeto'),
    ('Descripción de las actuaciones', 'descripcion'),
    ('Características de la Red Natura 2000 en la que se encuentran las parcelas de actuación', 'caracteristicas'),
    ('Efectos de las quemas prescritas sobre la RED NATURA 2000', 'efectos'),
    ('Afecciones a los habitats de interés comunitario presentes en el ZEC', 'afeccion_habitats'),
    ('Afecciones a los taxones de interés comunitario presentes en el ZEC', 'afeccion_taxones'),
    ('Conclusiones', 'conclusiones')
]

### HABITATS AFECTADOS

habitats_afectados = pd.read_sql('''
                SELECT distinct a.codue, a.nom_comun, generico, a.alianza, a.spsalianza
                FROM atlashabitats2005_extremadura a, quema_{}_todas b
                WHERE st_intersects(st_transform(b.geom, 23030), a.geom)
                '''.format(zona), conn)

##  HABITATS PARCELAS

habitats_parcelas = pd.read_sql('''
            WITH ab as (
                SELECT
                    a.nombre,
                    codue as codigo,
                    descripcio, 
                    st_area(st_transform(a.geom, 25830)) / 10000 as sup_parcela,
                    st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / 10000 as superficie,
                    st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / st_area(st_transform(a.geom, 23030)) * 100 as porcentaje
                FROM atlashabitats2005_extremadura b, quema_{}_todas a
                WHERE st_intersects(st_transform(a.geom, 23030), b.geom)
                GROUP BY a.nombre, b.codue, b. descripcio, a.parcela, b.geom, a.geom
                ORDER BY a.parcela
            )
            SELECT 
                ab.*,
                Sum(St_Area(St_Intersection(atlas.geom, St_Transform(zec.geom, 23030)))) / 10000 as sup_hab_zec,
                ab.superficie * 100 * 10000 / Sum(St_Area(St_Intersection(atlas.geom, St_Transform(zec.geom, 23030)))) as sup_hab_zec
            FROM ab, atlashabitats2005_extremadura atlas, sierra_gredos_y_valle_jerte zec
            WHERE st_intersects(atlas.geom, st_transform(zec.geom, 23030)) and ab.codigo = atlas.codue and ab.descripcio = atlas.descripcio
            GROUP BY ab.nombre, ab.codigo, ab.descripcio, ab.sup_parcela, ab.superficie, ab.porcentaje
            '''.format(zona), conn)

habitats_parcelas.columns = ['Parcela', 'Código UE', 'Descripción', 'Superficie Parcela', 'Superficie del hábitat en la parcela', 'Porcentaje de parcela ocupado por el hábitat', 'Superficie del hábitat en el ZEC', 'Porcentaje de la superficie del hábitat en la parcela sobre el total del hábitat en el ZEC']
# display(habitats_parcelas)

habitats_zec = pd.read_sql('''
            SELECT distinct a.codue, a.nom_comun, a.generico, nom_habita, alianza, spsalianza
            FROM atlashabitats2005_extremadura a, sierra_gredos_y_valle_jerte b
            WHERE st_intersects(st_transform(a.geom, 25830), b.geom)
            ''', conn)

habitats_zec = np.array_split(habitats_zec, 5)

## FICHAS

parcelas = pd.read_sql('''
            SELECT
                a.nombre,
                a.parcela,
                ST_Area(ST_Transform(a.geom, 25830)) / 10000 as sup, 
                ST_Perimeter(ST_Transform(a.geom, 25830)) as perim,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).min as pend_min,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).max as pend_max,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).mean as pend_mean,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).min as alt_min,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).max as alt_max,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).mean as alt_mean,
                (ST_SummaryStats(St_Union(ST_Clip(d.rast, st_transform(geom, 25830))))).mean as orient
            FROM 
                quema_{}_todas a,
                pendientes25 b,
                mdt25 c,
                orientaciones25 d
            GROUP BY
                a.nombre, a.parcela, a.geom
            ORDER BY a.parcela
            '''.format(zona), conn)

env = Environment(loader=FileSystemLoader('static/afeccion'))
template =env.get_template('base.html')
output = template.render(
    zona = zona,
    comarca = comarca,
    capitulos = capitulos,
    habitats_afectados = habitats_afectados,
    habitats = habitats_parcelas,
    habitats_zec = habitats_zec,
    parcelas = parcelas.to_dict(orient='records')
)

with open('static/afeccion/afeccion.html', 'w') as f:
    f.write(output)